In [1]:
import numpy as np
from pyfinite import ffield

In [2]:
#Final Matrices as obtained from Cryptanalysis.ipynb
At = [[84, 115, 8, 125, 97, 30, 17, 88], [0, 70, 28, 20, 43, 47, 120, 13], [0, 0, 43, 24, 3, 19, 18, 95], [0, 0, 0, 12, 110, 43, 101, 25], [0, 0, 0, 0, 112, 111, 2, 12], [0, 0, 0, 0, 0, 11, 92, 69], [0, 0, 0, 0, 0, 0, 27, 3], [0, 0, 0, 0, 0, 0, 0, 38]]
E = [19, 120, 40, 80, 88, 52, 22, 15]
A = [[At[j][i] for j in range(len(At))] for i in range(len(At[0]))]
A

[[84, 0, 0, 0, 0, 0, 0, 0],
 [115, 70, 0, 0, 0, 0, 0, 0],
 [8, 28, 43, 0, 0, 0, 0, 0],
 [125, 20, 24, 12, 0, 0, 0, 0],
 [97, 43, 3, 110, 112, 0, 0, 0],
 [30, 47, 19, 43, 111, 11, 0, 0],
 [17, 120, 18, 101, 2, 92, 27, 0],
 [88, 13, 95, 25, 12, 69, 3, 38]]

In [3]:
block_size = 8
F = ffield.FField(7, gen=0x83, useLUT=-1)
A = np.array((A))
augmentedA = np.zeros((block_size, block_size*2), dtype = int)
inverseA = np.zeros((block_size, block_size), dtype = int)
Einverse = np.zeros((128, 128), dtype = int)

exponents = []
for i in range(0,128):
    exponents.append([1])

for base in range(0,128):
    for exponent in range(1,127):
        temp = exponents[base][exponent-1]
        result = F.Multiply(temp, base)
        exponents[base]+=[result]

for base in range(0,128):
    for exponent in range(1,127):
        Einverse[exponent][exponents[base][exponent]] = base

inverses = [1]
for i in range(1, 128):
    inverses+=[F.Inverse(i)]
    assert F.Multiply(i, inverses[i]) == 1


for i in range(0,block_size):
    for j in range(0,block_size):
        augmentedA[i][j] = A[i][j]
    augmentedA[i][i+j+1] = 1

for j in range(0,block_size):
    assert np.any(augmentedA[j:,j] != 0) # assert pivot row exists: A is invertible
    pivot_row = np.where(augmentedA[j:,j] != 0)[0][0] + j
    augmentedA[[j, pivot_row]] = augmentedA[[pivot_row, j]]
    # make pivot k 1
    mul_fact = inverses[augmentedA[j][j]]
    for k in range(block_size*2):
        augmentedA[j][k] = F.Multiply(augmentedA[j][k], mul_fact)
    for i in range(0,block_size):
        if i!=j and augmentedA[i][j] != 0:
            mult_fact = augmentedA[i][j]
            for k in range(block_size*2):
                temp = F.Multiply(augmentedA[j][k], mult_fact)
                augmentedA[i][k] = F.Add(temp, augmentedA[i][k])

for i in range(0,block_size):
    for j in range(0,block_size):
        inverseA[i][j] = augmentedA[i][block_size+j]
print("A inverse matrix: \n{}".format(inverseA))

A inverse matrix: 
[[ 67   0   0   0   0   0   0   0]
 [ 80  77   0   0   0   0   0   0]
 [115 123  96   0   0   0   0   0]
 [ 87  33  67  94   0   0   0   0]
 [ 23 119 117  48  19   0   0   0]
 [ 13  48  84  87  48  74   0   0]
 [105  87  44  11  21  97  14   0]
 [108   7  88  48  83  31 121  56]]


In [4]:
password = "gsjgjljmlkfmigggmojufnipisjklimq" #Encrypted password
block_size = 16
num_blocks = int(len(password) / block_size) # 2 blocks

def Einverse_calculator(block, E):
    transformed = []
    for j in range(0,8):
        transformed+=[Einverse[E[j]][block[j]]]
    return transformed

def Ainverse_calculator(block, A):
    transformed = []
    for row_num in range(0,8):
        elem_sum = 0
        for col_num in range(0,8):
            elem = F.Multiply(A[row_num][col_num], block[col_num])
            elem_sum = F.Add(elem, elem_sum)
        transformed+=[elem_sum]
    return transformed

decrypted_password = ""
for i in range(0,2): # Decipher both the blocks of the password
    elements = password[block_size*i:block_size*(i+1)]
    currentBlock = []
    for j in range(0,8):
        currentBlock+=[(ord(elements[2*j]) - ord('f'))*16 + (ord(elements[2*j+1]) - ord('f'))]
    
    temp1 = Ainverse_calculator(Einverse_calculator(currentBlock, E), inverseA)
    EA = Einverse_calculator(temp1, E)
    
    temp2 = Ainverse_calculator(EA, inverseA)
    EAEAE = Einverse_calculator(temp2, E)
    
    print("Inverse EAEAE values for block:",str(i)+": ",EAEAE)
    for ch in EAEAE:
        decrypted_password += chr(ch)
print("The password is",decrypted_password)
    

Inverse EAEAE values for block: 0:  [115, 121, 113, 118, 122, 121, 111, 108]
Inverse EAEAE values for block: 1:  [100, 109, 48, 48, 48, 48, 48, 48]
The password is syqvzyoldm000000
